In [1]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [68]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [3]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [8]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[1.6011e+02, 1.1000e+00],
         [5.6049e+01, 1.4410e-02],
         [6.5039e+01, 3.7226e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.0301e+02, 1.1000e+00],
         [1.4704e+02, 2.0000e-02],
         [1.7301e+02, 6.3000e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[2.8915e+02, 1.1000e+00],
         [7.7038e+01, 5.4000e-02],
         [7.8043e+01, 6.0000e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[1.5207e+02, 1.1000e+00],
         [1.1006e+02, 1.0000e+00],
         [1.1104e+02, 1.2939e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.4617e+02, 1.1000e+00],
         [6.0576e+01, 1.

In [9]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])


In [10]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [11]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [12]:
tmp_original['precursor_mz']

tensor([160.1121, 303.0100, 289.1547, 292.0654, 413.1730, 120.0700, 351.2166,
        383.1100, 123.0440, 152.0706, 346.1650, 287.0900])

In [13]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+'],)

In [14]:
tmp_original['mol_freq']

tensor([ 38., 322.,  26.,  34.,  15.,  77.,   8.,   8.,   7.,  83.,   2.,  33.])

In [15]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [16]:
sum(tmp_original['labels'])

tensor(12)

In [17]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1536, 1792, 2048, 2304, 2560, 2816])

In [18]:
len(tmp_original['labels'])

3072

In [19]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])

In [20]:
sum(tmp_original['batch_ptr'])

tensor(3072)

In [21]:
len(tmp_original['candidates'])

3072

In [22]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [23]:
len(tmp_original['candidates_smiles'])

3072

# MSnRetrieval

In [69]:
from massspecgym.featurize import SpectrumFeaturizer

In [70]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [71]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [72]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [73]:
# file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/min_sample_trees.mgf"


In [74]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 16476
MSnRetrievalDataset length: 16476
Precomputing candidate-side transformations using multiprocessing with chunking...
Using 8 processes for precomputation.


Processing chunks: 100%|██████████| 33/33 [41:23<00:00, 75.26s/it]


Precomputation complete. Cache saved to /Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.h5


In [76]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [77]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 12536
Val dataset size: 1952


In [78]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

[PROFILE] __getitem__ took 0.532142 seconds
[PROFILE] __getitem__ took 0.031780 seconds
[PROFILE] __getitem__ took 0.037800 seconds
[PROFILE] __getitem__ took 0.038638 seconds
[PROFILE] __getitem__ took 0.027755 seconds
[PROFILE] __getitem__ took 0.035815 seconds
[PROFILE] __getitem__ took 0.040279 seconds
[PROFILE] __getitem__ took 0.039242 seconds
[PROFILE] __getitem__ took 0.041408 seconds
[PROFILE] __getitem__ took 0.040312 seconds
[PROFILE] __getitem__ took 0.038661 seconds
[PROFILE] __getitem__ took 0.025656 seconds
[PROFILE] collate_fn took 0.006148 seconds
{'spec': DataBatch(x=[141, 1039], edge_index=[2, 258], batch=[141], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([949.5002, 574.2258, 269.0808, 405.2748, 589.1893, 436.1059, 324.

In [79]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([ 25,  80, 256, 256,  45, 256, 256, 256, 256, 256, 256,  10],
       dtype=torch.int32)


In [80]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

Data types in tmp_original:
  spec: <class 'torch.Tensor'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  identifier: <class 'list'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>
Data types in tmp_msn:
  spec: <class 'abc.DataBatch'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  identifier: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [81]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [82]:
tmp_msn['spec'] 

DataBatch(x=[141, 1039], edge_index=[2, 258], batch=[141], ptr=[13])

In [83]:
tmp_msn['precursor_mz']

tensor([949.5002, 574.2258, 269.0808, 405.2748, 589.1893, 436.1059, 324.1958,
        216.0655, 322.1332, 469.2001, 244.0903, 850.5020])

In [84]:
tmp_msn['mol_freq']

tensor([2., 1., 1., 2., 1., 1., 3., 1., 1., 1., 1., 3.])

In [85]:
tmp_msn['identifier']

['0001594_0000000',
 '0081112_0000000',
 '0076845_0000000',
 '0081351_0000000',
 '0100014_0000000',
 '0075818_0000000',
 '0071113_0000000',
 '0084945_0000000',
 '0022953_0000000',
 '0032029_0000000',
 '0087941_0000000',
 '0006315_0000000']

In [86]:
tmp_msn['adduct']

['[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+NH4]+']

In [87]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [88]:
tmp_msn['smiles']

['C[C@@H]1C[C@@H]([C@@]2([C@@H]([C@@]1(C)CC[C@@](C)(C=C)O[C@H]3[C@@H]([C@H]([C@H]([C@H](O3)C)O[C@H]4[C@@H]([C@@H]([C@H]([C@@H](O4)C)O)O)O)O)O)CCC=C2C)C)O[C@H]5[C@@H]([C@@H]([C@H]([C@@H](O5)C)O[C@H]6[C@@H]([C@H]([C@@H]([C@H](O6)COC(=O)C)O)O)O)O)O',
 'CC(C)C1=CC=CC=C1CC2=CC(=C(C(=C2O)O)O)C(=O)NC3=CC=C(C=C3)S(=O)(=O)C4=CC=CC=C4C(C)(C)C',
 'COC1=CC=CC(=C1)C2=C(C(=O)C3=CC=CC=C3O2)O',
 'CCCCCCCC(=O)N[C@H](CN1CCCC1)[C@@H](C2=CC3=C(C=C2)OCCO3)O',
 'CC(C)N1C=C(C(=O)N(C1=O)C2=CC=C(C=C2)F)C(=O)NC3=CC(=C(C=C3)OC4=C5C=C(C(=CC5=NC=C4)OC)OC)F',
 'C1=CC=C(C(=C1)C(C2=CC(=C3C=CC=NC3=C2O)Cl)NC4=CC=CC=N4)OCC(=O)O',
 'CCN1CCCC(C1)OC(=O)C(C2=CC=CC=C2)C3=CC=CC=C3',
 'COC1=C2C=COC2=NC3=C1C=CC=C3O',
 'CC1=NC2=C(C=CN=C2)C(=N1)N3CC(C3)CN(C)S(=O)(=O)C',
 'CN1CCN(CC1)C(=O)C2CCN(CC2)C(=O)C3=CC4=C(N3CC5=CC=C(C=C5)Cl)C=CO4',
 'CCCC1=NC2=C(S1)C3=CC=CC=C3N=C2N',
 'CC(C)C(C(=O)OC(C)C(=O)NC(C(C)C)C(=O)OC(C(C)C)C(=O)NC(C(C)C)C(=O)OCC(CO)O)NC(=O)C(C)OC(=O)C(C(C)C)NC(=O)C(C(C)C)O']

In [89]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [90]:
sum(tmp_msn['labels'])

tensor(12)

In [91]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,   25,  105,  361,  617,  662,  918, 1174, 1430, 1686, 1942, 2198])

In [92]:
len(tmp_msn['labels'])

2208

In [93]:
tmp_msn['batch_ptr']

tensor([ 25,  80, 256, 256,  45, 256, 256, 256, 256, 256, 256,  10],
       dtype=torch.int32)

In [94]:
sum(tmp_msn['batch_ptr'])

tensor(2208, dtype=torch.int32)

In [95]:
len(tmp_msn['candidates'])

2208

In [96]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [97]:
tmp_msn.keys() == tmp_original.keys()

True

In [98]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [99]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [100]:
tmp_original['batch_ptr'].dtype

torch.int64

In [101]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [102]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2208, dtype=torch.int32), 2208)

In [103]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(3072), 3072)

In [104]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [105]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [106]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [107]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [108]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [109]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [110]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [111]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [66]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [67]:
len(tmp_msn['candidates_smiles'])

2598